In [2]:
#Connecting to google drive to access the files
from google.colab import drive,files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Creating a CNN model for 10 class digital recognition problem

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class MyCNNNetwork(nn.Module):
#defining network model
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, kernel_size=(5,5))
    self.pool=nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,kernel_size=(5,5))
    self.fc1 = nn.Linear(16*4*4,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)


#defining forward method
  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=x.view(-1,self.num_flat_features(x))
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=F.log_softmax(self.fc3(x))
    return x

#falttening features to send as input to Fully COnnected Feed Forward Network
  def num_flat_features(self,x):
    size=x.size()[1:]
    num_features=1
    for s in size:
      num_features*=s
    return num_features


#Printing the network structure
net=MyCNNNetwork().to(device)
print(net)


MyCNNNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params=list(net.parameters())
for i in range(len(params)):
  print(params[i].size())

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 256])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [5]:
# define a customised dataset in torch
import os
import glob
import numpy as np
from skimage import io

from torch.utils.data import Dataset,DataLoader

#overriding : __init__,__len__, and __getitem__ methods

class MNISTDatasets(Dataset):

  def __init__(self,dir,transform=None):
    self.dir=dir # for example: /content/drive/My Drive/trainingset/1 
    self.transform=transform

  def __len__(self):
    files=glob.glob(self.dir+'/*.jpg')[:1000]
    return len(files)

  def __getitem__(self,idx):
    if torch.is_tensor(idx):
      idx=idx.tolist()

    all_files=glob.glob(self.dir+'/*.jpg')[:1000]
    img_fname=os.path.join(self.dir,all_files[idx])
    image=io.imread(img_fname)

    digit=int(self.dir.split('/')[-1].strip())
    label=np.array(digit)
    
    sample={'image':image,'label':label}

    if self.transform:
      instance=self.transform(sample)   
    
    return instance

 

In [6]:
#cutomised transformation with several operations:
# Rescale, ToTensor
from skimage import transform
from torchvision import transforms,utils

class Rescale(object):

  def __init__(self,output_size):
    assert isinstance(output_size,(int,tuple))
    self.output_size=output_size

  def __call__(self,sample):
    image,label=sample['image'],sample['label']    
    h,w=image.shape[-2:]


    if isinstance(self.output_size,int):
      if h>w:
        new_h,new_w=self.output_size*h/w,self.output_size
      else:
          new_h,new_w=self.output_size,self.output_size*w/h
    else:
      new_h,new_w=self.output_size
    
    new_h,new_w=int(new_h),int(new_w)

    new_img=transform.resize(image,(new_h,new_w))
    
    return {'image':new_img,'label':label}


class ToTensor(object):

  def __call__(self,sample):
    image,label=sample['image'],sample['label']

    image=image.reshape((1,image.shape[0],image.shape[1]))

    
    return {'image':torch.from_numpy(image),'label':torch.from_numpy(label)}




In [7]:
from torch.utils.data import random_split

batch_size=32

list_datasets=[]
for i in range(10):

  cur_ds=MNISTDatasets(dir='/content/drive/My Drive/trainingset/'+str(i),transform=transforms.Compose([Rescale(28),ToTensor()]))
  list_datasets.append(cur_ds)

dataset=torch.utils.data.ConcatDataset(list_datasets)
print(len(dataset))

train_size=int(len(dataset)*0.7)
val_size=len(dataset)-train_size
train_dataset,val_dataset=random_split(dataset,[train_size,val_size])

train_loader=DataLoader(train_dataset,batch_size,shuffle=True,num_workers=1)
val_loader=DataLoader(val_dataset,batch_size,shuffle=True,num_workers=1)

10000


In [8]:
#Training code

epochs=5
learning_rate=1e-2
optimizer=optim.Adam(net.parameters(),lr=learning_rate,weight_decay=1e-5)
criterion=nn.CrossEntropyLoss()

for epoch in range(epochs):
  
  net.train()

  running_loss=0.0
  
  for i,batch in enumerate(train_loader):
    inputs,targets=batch['image'].to(device,dtype=torch.float),batch['label'].to(device,dtype=torch.long)
    optimizer.zero_grad()
    predicted_outputs=net(inputs)
    loss=criterion(predicted_outputs,targets)
    loss.backward()
    optimizer.step()

    running_loss+=loss.item()
    #training loss for every 10 batches
    if(i+1)%10==0:
      print('epoch %d,batch: %d,training loss: %.3f' % (epoch+1,i+1,running_loss/10))
      running_loss=0.0

#validation code

  net.eval()

  correct=[0.0]*10
  total=[0.]*10

  with torch.no_grad():
    for b,data in enumerate(val_loader):
      images,labels=data['image'].to(device,dtype=torch.float),data['label'].to(device,dtype=torch.long)
      predicted_outputs=net(images)
      _,predicted=torch.max(predicted_outputs,1)
      c=(predicted==labels)
      for i in range(len(labels)):
        label=labels[i]
        correct[label]+=c[i].item()
        total[label]+=1
#accuracy for the entire validation set per epochh
  for i in range(10):
        print('\tValidation accuracy for digit %d:%.2f'% (i,100*correct[i]/total[i]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 1,batch: 10,training loss: 2.191
epoch 1,batch: 20,training loss: 1.638
epoch 1,batch: 30,training loss: 1.089
epoch 1,batch: 40,training loss: 0.706
epoch 1,batch: 50,training loss: 0.551
epoch 1,batch: 60,training loss: 0.567
epoch 1,batch: 70,training loss: 0.440
epoch 1,batch: 80,training loss: 0.378
epoch 1,batch: 90,training loss: 0.490
epoch 1,batch: 100,training loss: 0.345
epoch 1,batch: 110,training loss: 0.324
epoch 1,batch: 120,training loss: 0.375
epoch 1,batch: 130,training loss: 0.315
epoch 1,batch: 140,training loss: 0.219
epoch 1,batch: 150,training loss: 0.273
epoch 1,batch: 160,training loss: 0.251
epoch 1,batch: 170,training loss: 0.220
epoch 1,batch: 180,training loss: 0.216
epoch 1,batch: 190,training loss: 0.194
epoch 1,batch: 200,training loss: 0.293
epoch 1,batch: 210,training loss: 0.252
	Validation accuracy for digit 0:98.34
	Validation accuracy for digit 1:94.62
	Validation accuracy for digit 2:89.71
	Validation accuracy for digit 3:94.81
	Validation a